In [3]:
import pandas as pd
from sympy import symbols
from sympy.logic.boolalg import And, Or, Not, simplify_logic, BooleanTrue, BooleanFalse

# Define boolean variables
A, R, E = symbols('A R E')

def dnf_expr_from_8bit(vector_str: str):
    """
    Return a simplified SymPy DNF *expression* for an 8-bit vector.
    Bit order: DB_mean, DB_nice, DG_mean, DG_nice, CB_mean, CB_nice, CG_mean, CG_nice
    Index mapping: i = 4*A + 2*R + E  with A=0/1(Defect/Cooperate), R=0/1(Bad/Good), E=0/1(Mean/Nice)
    """
    s = ''.join(ch for ch in str(vector_str).strip() if ch in '01')
    if len(s) != 8:
        return BooleanFalse

    bits = [int(b) for b in s]
    terms = []
    for i, b in enumerate(bits):
        if b == 1:
            a = (i >> 2) & 1
            r = (i >> 1) & 1
            e =  i       & 1
            terms.append(And(A if a else Not(A),
                             R if r else Not(R),
                             E if e else Not(E)))
    if not terms:
        return BooleanFalse
    return simplify_logic(Or(*terms), form='dnf', force=True)

def count_literals_expr(expr) -> int:
    """Count literal occurrences in a SymPy boolean expression in DNF."""
    if expr is BooleanFalse:
        return 0
    if expr is BooleanTrue:
        return 1

    count = 0
    def recurse(node):
        nonlocal count
        if node.is_Symbol:
            count += 1
        elif isinstance(node, Not) and node.args and node.args[0].is_Symbol:
            count += 1
        else:
            for arg in getattr(node, 'args', ()):
                recurse(arg)
    recurse(expr)
    return count


def emotion_leniency(v: str) -> float:
    """
    Compute emotion leniency for an 8-bit social norm vector.
    Bit order: DB_mean, DB_nice, DG_mean, DG_nice, CB_mean, CB_nice, CG_mean, CG_nice.
    """
    v = ''.join(ch for ch in str(v).strip() if ch in '01')
    if len(v) != 8:
        return np.nan

    pairs = [(0, 1), (2, 3), (4, 5), (6, 7)]
    equal_bits = sum(int(v[i]) == int(v[j]) for i, j in pairs)
    return equal_bits / 4



# ---------- CONFIG ----------
OUTPUT_CSV = "data/all_8bit_norms_with_dnf.csv"
# ----------------------------

# Generate all 256 binary strings
vectors = [format(i, "08b") for i in range(256)]

# Compute DNF and literal counts
records = []
for v in vectors:
    expr = dnf_expr_from_8bit(v)
    literals = count_literals_expr(expr)
    records.append({
        "8bit_vector": v,
        "DNF": str(expr),
        "DNF_literals": literals
    })

# Convert to DataFrame
df = pd.DataFrame(records)
df["Emotion_Leniency"] = df["8bit_vector"].apply(emotion_leniency)


# Save to CSV
df.to_csv(OUTPUT_CSV, index=False)
print(f"Generated all 256 8-bit norms with DNF + literal counts → {OUTPUT_CSV}")


Generated all 256 8-bit norms with DNF + literal counts → data/all_8bit_norms_with_dnf.csv
